In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:,.4f}'.format)

# <font color='navy'> Datos

In [2]:
df = pd.read_csv('losses_repaso.csv')
ecl =  801_540
std =  455_640
var =  2_173_1000 
cvar =  2_403_500

#ecl = 392_510
#std = 204_580
#var = 989_700
#cvar = 1_101_200 

ucl = var - ecl

# <font color='navy'> Tranches

In [3]:
port = 10_915_382
equity = ecl * 0.5
mezanine = ecl * 0.75
junior = ecl * 1

#port = 5_552_048
#equity = ecl
#mezanine = ecl * 1.2
#junior = ecl * 1.3

senior = port - ecl - equity - mezanine - junior

In [4]:
lim_ecl = ecl
lim_equity = ecl + equity
lim_mezanine = ecl + equity + mezanine
lim_junior = ecl + equity + mezanine + junior
lim_senior = ecl + equity + mezanine + junior + senior

# <font color='navy'> Grupos

In [5]:
maximo = df.max().values[0]
minimo = df.min().values[0]
rango = maximo - minimo

In [6]:
df_grupos = pd.DataFrame({
    'grupos': list(range(1, 101)),
})
df_grupos['perdida'] = minimo + rango/99 * df_grupos.index

In [7]:
losses = df['losses'].to_numpy()
bins = df_grupos['perdida'].to_numpy()

frecuencias = []

for i, limite in enumerate(bins):
    if i == 0:
        count = np.sum(losses <= limite)
    else:
        count = np.sum((losses > bins[i-1]) & (losses <= limite))
    frecuencias.append(count)

df_grupos['frecuencia'] = frecuencias
df_grupos['prob_marginal'] = df_grupos['frecuencia'] / df_grupos['frecuencia'].sum()
df_grupos['prob_acumulada'] = df_grupos['prob_marginal'].cumsum()
df_grupos

,grupos,perdida,frecuencia,prob_marginal,prob_acumulada
0,1,0.0000,20,0.0000,0.0000
1,2,"37,885.7747",277,0.0003,0.0003
2,3,"75,771.5494",1419,0.0014,0.0017
3,4,"113,657.3240",3765,0.0038,0.0055
4,5,"151,543.0987",7417,0.0074,0.0129
...,...,...,...,...,...
95,96,"3,599,148.5943",0,0.0000,1.0000
96,97,"3,637,034.3690",2,0.0000,1.0000
97,98,"3,674,920.1436",0,0.0000,1.0000
98,99,"3,712,805.9183",2,0.0000,1.0000


# <font color='navy'> Cortes

In [8]:
df_filtered = df_grupos[df_grupos['perdida'] < lim_ecl]
df_extra = df_grupos[df_grupos['perdida'] >= lim_ecl].head(1)
df_ecl = pd.concat([df_filtered, df_extra])

df_filtered_equity = df_grupos[(df_grupos['perdida'] > df_ecl['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_equity)]
df_extra_equity = df_grupos[df_grupos['perdida'] >= lim_equity].head(1)
df_equity = pd.concat([df_filtered_equity, df_extra_equity])

df_filtered_mezanine = df_grupos[(df_grupos['perdida'] > df_equity['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_mezanine)]
df_extra_mezanine = df_grupos[df_grupos['perdida'] >= lim_mezanine].head(1)
df_mezanine = pd.concat([df_filtered_mezanine, df_extra_mezanine])

df_filtered_junior = df_grupos[(df_grupos['perdida'] > df_mezanine['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_junior)]
df_extra_junior = df_grupos[df_grupos['perdida'] >= lim_junior].head(1)
df_junior = pd.concat([df_filtered_junior, df_extra_junior])

df_filtered_senior = df_grupos[(df_grupos['perdida'] > df_junior['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_senior)]
df_extra_senior = df_grupos[df_grupos['perdida'] >= lim_senior].head(1)
df_senior = pd.concat([df_filtered_senior, df_extra_senior])

# <font color='navy'> Métricas

In [9]:
equity_loss_prob = 1 - df_equity['prob_acumulada'].iloc[0]
mezanine_loss_prob = 1 - df_mezanine['prob_acumulada'].iloc[0]
junior_loss_prob = 1 - df_junior['prob_acumulada'].iloc[0]

if len(df_senior) > 0:
    senior_loss_prob = 1 - df_senior['prob_acumulada'].iloc[0]
else:
    senior_loss_prob = 0

In [10]:
equity_loss = np.sum((df_equity['perdida'] - df_ecl['perdida'].iloc[-1]) * df_equity['prob_marginal'])
mezanine_loss = np.sum((df_mezanine['perdida'] - df_equity['perdida'].iloc[-1]) * df_mezanine['prob_marginal'])
junior_loss = np.sum((df_junior['perdida'] - df_mezanine['perdida'].iloc[-1]) * df_junior['prob_marginal'])
senior_loss = np.sum((df_senior['perdida'] - df_junior['perdida'].iloc[-1]) * df_senior['prob_marginal'])

In [11]:
expected_loss = np.array([equity_loss, mezanine_loss, junior_loss, senior_loss])
port_values = np.array([equity, mezanine, junior, senior])
spread = expected_loss / port_values

In [12]:
df_resultados = pd.DataFrame({
    'tramo': ['equity', 'mezanine', 'junior', 'senior'],
    'probabilidad': [equity_loss_prob, mezanine_loss_prob, junior_loss_prob, senior_loss_prob],
    'pérdida esperada': expected_loss,
    'spread': spread
})
df_resultados['probabilidad'] = df_resultados['probabilidad'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados['spread'] = df_resultados['spread'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados

,tramo,probabilidad,pérdida esperada,spread
0,equity,35.85%,"39,746.2314",9.92%
1,mezanine,16.06%,"36,439.7883",6.06%
2,junior,3.11%,"8,794.1597",1.10%
3,senior,0.12%,289.1442,0.00%
